In [1]:
import datetime
import os
from collections import OrderedDict
import pprint
pp = pprint.PrettyPrinter(indent=4)


In [2]:
import pandas as pd

In [3]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook, reset_output
from bokeh.layouts import gridplot
from bokeh.models import Arrow, NormalHead, OpenHead, VeeHead
from bokeh.models import Label
from bokeh.models import Span
from bokeh.embed import components


In [4]:
import scipy.optimize as optim
import numpy as np
from scipy import stats
import statsmodels.api as sm
import random
from scipy.stats import binom
from scipy.stats import ttest_ind_from_stats
from scipy.stats import ttest_ind



In [5]:
output_notebook()

Loading BokehJS ...

In [6]:
def get_state_mask_start():
    return  [('Alabama', datetime.datetime(2020, 7, 16)),
             ('Arizona', datetime.datetime(2020,6,17)),
             ('California', datetime.datetime(2020, 6, 18)),
             ('Connecticut', datetime.datetime(2020,4, 20)),
            ('Delaware', datetime.datetime(2020, 4, 28)),
            ('Hawaii', datetime.datetime(2020, 4, 20)),
            ('Illinois', datetime.datetime(2020,5,1)),
            ('Kansas', datetime.datetime(2020, 7, 3)),
            ('Kentucky', datetime.datetime(2020, 7, 10)),
            ('Maine', datetime.datetime(2020, 5, 1)),
            ('Maryland', datetime.datetime(2020, 4, 18)),
            ('Massachusetts', datetime.datetime(2020, 5, 6)),
            ('Michigan', datetime.datetime(2020, 6, 18)),
            ('Nevada', datetime.datetime(2020, 6, 24)),
            ('New Jersey', datetime.datetime(2020, 4, 8)),
            ('New Mexico', datetime.datetime(2020, 5, 16)),
            ('New York', datetime.datetime(2020, 4, 17)),
            ('North Carolina', datetime.datetime(2020, 6, 26)),
            ('Oregon', datetime.datetime(2020, 7, 1)),
            ('Pennsylvania', datetime.datetime(2020, 4, 19)),
            ('Rhode Island', datetime.datetime(2020, 5, 18)),
            ('Texas', datetime.datetime(2020, 7, 3)),
            ('Virginia', datetime.datetime(2020, 5, 29)),
            ('Washington', datetime.datetime(2020, 6, 26)),
            ('West Virginia', datetime.datetime(2020, 7, 6)),
           ]

In [7]:
def get_state_mask_start_with_after():
    return  [            ('Alabama', datetime.datetime(2020, 7, 16), datetime.datetime(2020,10, 15),
                         datetime.datetime(2020,10, 30)),
             ('Arizona', datetime.datetime(2020,6,17), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
             ('California', datetime.datetime(2020, 6, 18), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
           ('Connecticut', datetime.datetime(2020,4, 20), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Delaware', datetime.datetime(2020, 4, 28), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Hawaii', datetime.datetime(2020, 4, 20), datetime.datetime(2020,8, 20),datetime.datetime(2020,9, 4)),
            ('Illinois', datetime.datetime(2020,5,1), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Kansas', datetime.datetime(2020, 7, 3), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Kentucky', datetime.datetime(2020, 7, 10), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Maine', datetime.datetime(2020, 5, 1), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Maryland', datetime.datetime(2020, 4, 18), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Massachusetts', datetime.datetime(2020, 5, 6), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Michigan', datetime.datetime(2020, 6, 18), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Nevada', datetime.datetime(2020, 6, 24), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('New Jersey', datetime.datetime(2020, 4, 8),datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('New Mexico', datetime.datetime(2020, 5, 16), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('New York', datetime.datetime(2020, 4, 17), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('North Carolina', datetime.datetime(2020, 6, 26), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Oregon', datetime.datetime(2020, 7, 1), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Pennsylvania', datetime.datetime(2020, 4, 19), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Rhode Island', datetime.datetime(2020, 5, 18), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Texas', datetime.datetime(2020, 7, 3), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Virginia', datetime.datetime(2020, 5, 29), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('Washington', datetime.datetime(2020, 6, 26), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
            ('West Virginia', datetime.datetime(2020, 7, 6), datetime.datetime(2020,10, 15),datetime.datetime(2020,10, 30)),
           ]

In [8]:
"""
Make a bar graph
"""
def make_mask_graph(df, mask_start, title = None, plot_height = 450, plot_width = 450,
                   incubation_period = 5, window = 3):
    labels = df['date'].tolist()
    nums = df['cases'].rolling(window).mean()
    p = figure(x_axis_type = 'datetime', title = title, 
                 plot_width = plot_width , plot_height = plot_height, y_range = None)
    dd = list(zip(labels, nums))
    bef = [x for x in dd if x[0] <= mask_start]

    incc = [x for x in dd if x[0] >  mask_start 
        and x[0] <= mask_start + datetime.timedelta(days = incubation_period)
      ]
    aff = [x for x in dd if x[0] > mask_start + datetime.timedelta(days = incubation_period)]

    p.vbar(x=[x[0] for x in bef], top=[x[1] for x in bef] , line_width = 5, 
           width = .9, color = 'yellow', 
       )
    p.vbar(x=[x[0] for x in incc], top=[x[1] for x in incc] , line_width = 5, 
           width = .9, color = 'red', 
       )
    p.vbar(x=[x[0] for x in aff], top=[x[1] for x in aff] , line_width = 5, 
           width = .9, color = 'orange', 
       )
    p.legend.location = "top_center"
    return p

Create a graphs with 3 time period: one before the mask mandate, one for incubation(5 days) and one after the 
incubation period.

In [9]:
reset_output()
from bokeh.models import DatetimeTickFormatter

def do_mask_mandates(window = 3, plot_height = 450, 
                    plot_width = 450, ncols = 3):
    grids = []
    start_date = datetime.datetime(2020,3, 1)
    df_all = pd.read_csv('data/states.csv')
    df_all['date'] = pd.to_datetime(df_all['date'])
    for i in get_state_mask_start():
        df = df_all[(df_all['state'] == i[0]) & (df_all['date'] >= start_date)]
        p =  make_mask_graph(df = df,
                 mask_start = i[1],
                title = i[0], plot_height = plot_height,
                plot_width = plot_width,
                window = window,
               )
        p.xaxis.formatter=DatetimeTickFormatter(days="%m", months = '%m')

        grids.append(p)
    show(gridplot(grids, ncols = ncols))
do_mask_mandates(window = 7, plot_height = 150, plot_width = 150, ncols=6)

/home/henry/Envs2/covid19/lib/python3.6/site-packages/bokeh/models/plots.py:764: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


In [10]:
"""
Make a bar graph
"""
def make_mask_graph_with_after(df, mask_start, after, after_end,
            title = None, plot_height = 450, plot_width = 450,
                   incubation_period = 5, window = 3):
    labels = df['date'].tolist()
    nums = df['cases'].rolling(window).mean()
    p = figure(x_axis_type = 'datetime', title = title, 
                 plot_width = plot_width , plot_height = plot_height, y_range = None)
    dd = list(zip(labels, nums))
    bef = [x for x in dd if x[0] <= mask_start]

    incc = [x for x in dd if x[0] >  mask_start 
        and x[0] <= mask_start + datetime.timedelta(days = incubation_period)
      ]
    aff = [x for x in dd if x[0] > mask_start + datetime.timedelta(days = incubation_period)
           and x[0] < after
          ]
    increase = [x for x in dd if x[0] > after and x[0] <= after_end]
    after_increase = [x for x in dd if x[0] > after_end ]

    p.vbar(x=[x[0] for x in bef], top=[x[1] for x in bef] , line_width = 5, 
           width = .9, color = 'yellow', 
       )
    p.vbar(x=[x[0] for x in incc], top=[x[1] for x in incc] , line_width = 5, 
           width = .9, color = 'red', 
       )
    p.vbar(x=[x[0] for x in aff], top=[x[1] for x in aff] , line_width = 5, 
           width = .9, color = 'orange', 
       )
    p.vbar(x=[x[0] for x in increase], top=[x[1] for x in increase] , line_width = 5, 
           width = .9, color = 'pink', 
       )
    p.vbar(x=[x[0] for x in after_increase], top=[x[1] for x in after_increase] , line_width = 5, 
           width = .9, color = 'orange', 
       )
    p.legend.location = "top_center"
    return p

In [11]:
reset_output()
from bokeh.models import DatetimeTickFormatter
def do_mask_mandates_with_after(window = 3, plot_height = 450, 
                    plot_width = 450, ncols = 3):
    grids = []
    start_date = datetime.datetime(2020,3, 1)
    df_all = pd.read_csv('data/states.csv')
    df_all['date'] = pd.to_datetime(df_all['date'])
    for i in get_state_mask_start_with_after():
        df = df_all[(df_all['state'] == i[0]) & (df_all['date'] >= start_date)]
        p =  make_mask_graph_with_after(df = df,
                 mask_start = i[1],
                after = i[2],
                after_end = i[3],
                title = i[0], plot_height = plot_height,
                plot_width = plot_width,
                window = window,
               )
        p.xaxis.formatter=DatetimeTickFormatter(days="%m", months = '%m')
        grids.append(p)
    show(gridplot(grids, ncols = ncols))
do_mask_mandates_with_after(window = 7, plot_height = 400, plot_width = 400, ncols=3)

/home/henry/Envs2/covid19/lib/python3.6/site-packages/bokeh/models/plots.py:764: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


Note how the pattern appears random. In some cases, the period after the incubation shows decrease (the desired
outcome). But in others, the number of cases increases. In still ohters, the cases decrease, but then later 
greatly increase.

In [12]:
def exp_func(x, initial, ratio):
    return initial * np.power(ratio, x - 1)

In [13]:
output_notebook()

Loading BokehJS ...

In [14]:
def states_poisson_graph(df, state, mask_start, plot_width = 350, plot_height = 350,
        title = None):
    reset_output()
    df_wash=df[df['state'] == state]
    p = figure(x_axis_type = 'datetime', plot_width = plot_width, 
               plot_height = plot_height, title = title)
    mask_start_span = Span(location=mask_start,
                            dimension='height', line_color='red',
                            line_dash='dashed', line_width=1)
    p.line(x = df_wash['date'], y = df_wash['cases'])
    p.add_layout(mask_start_span)
    p.yaxis.axis_label = 'cases'
    return p


def all_states_poisson_graph():
    df = pd.read_csv('data/state_poisson.csv')
    df['date'] =pd.to_datetime(df['date'])
    grid = []
    for i in get_state_mask_start():
        grid.append(states_poisson_graph(df, i[0], i[1],title = i[0]))
    return grid
grids =all_states_poisson_graph()
grid = gridplot(grids, ncols = 3)
show(grid)


In [15]:
def get_linear(x, y):
    X = list(zip(*[x]))
    xm = sm.add_constant(X)
    model = sm.OLS(y, xm) 
    result = model.fit()
    return result.params[1], result.pvalues[1], result.predict(xm)

In [16]:
def get_ssr(y, y_hat):
    final = []
    for i in range(len(y)):
        final.append((y[i] - y_hat[i])**2)
    return final


In [17]:
def ssr_is_sig(y, y_hat_lin, y_hat_exp):
    ssr_lin = get_ssr(y, y_hat_lin)
    ssr_exp = get_ssr(y, y_hat_exp)
    resamp_res = repeat_resample_sum(ssr_exp, ssr_lin)
    p_value = 1 - len([x for x in resamp_res if x < 0])/len(resamp_res)
    return p_value < .05/2


In [18]:
def exp_is_sig(x, y):
    ratios, y_hat_low, y_hat_high = get_ratios(x = x, y=y, num_iter = 100)
    l = np.quantile(ratios, .05/2)
    h = np.percentile(ratios, 97.5)
    return not l <= 1 <= h


In [19]:
def exp_p_value(x, y):
    ratios, y_hat_low, y_hat_high = get_ratios(x = x, y=y, num_iter = 100)
    med = np.median(ratios)
    if med < 1:
        p = len([x for x in ratios if x > 1])/len(ratios)
    else:
        p = len([x for x in ratios if x < 1])/len(ratios)

    return p * 2

In [20]:
def resample(l):
    final = []
    for i in range(len(l)):
        final.append(random.choice(l))
    return final

def repeat_resample_sum(sample_a, sample_b, num_iter = 1000):
    difference_in_sums = []
    for i in range(num_iter):
        resample_a = resample(sample_a)
        resample_b = resample(sample_b)
        difference_in_sums.append(sum(resample_a) - sum(resample_b))
    return difference_in_sums

def repeat_resample(sample, num_iter = 1000):
    means = []
    for i in range(num_iter):
        resamp = resample(sample)
        means.append(np.mean(resamp))
    return means

In [21]:
def get_ratios(x, y, num_iter = 100):
    zip_obj = list(zip(x, y))
    ratios = []
    all_lines = []
    for i in range(num_iter):
        new_ = resample(zip_obj)
        new_ = sorted(new_, key = lambda x: x[0])
        x_ = [x[0] for x in new_]
        y_ = [x[1] for x in new_]
        try:
            popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(x_), ydata = np.array(y_))
        except RuntimeError:
            continue
        y_hat = [exp_func(initial = popt[0], ratio = popt[1], x = x) for x in x]
        all_lines.append(y_hat)
        ratios.append(popt[1])
    points = zip(*all_lines)
    high = []
    low = []
    for i in points:
        high.append(np.percentile(i, 95))
        low.append(np.percentile(i, 5))
    return ratios, low, high

In [22]:
def print_results(the_dict):
    def get_1_spacer(state):
        if len(state) < 8:
            return '\t'
        return ''
    print('state\t\tBefore\tAfter\tDiff')
    print('=====\t\t======\t=====\t====')

    for key in sorted(the_dict.keys()):
        pre = the_dict[key]['pre']['exp_ratio']
        post = the_dict[key]['post']['exp_ratio']
        diff = post - pre
        print('{state}\t{spacer1}{pre}\t{post}\t{diff}'.format(
            state = key, 
            pre =f'{pre:5.2f}',
            post  =f'{post:5.2f}',
            spacer1 = get_1_spacer(key),
            diff = f'{diff:5.2f}',
             
             ))

In [23]:
 def get_fit_info(cases):
        final = {}
        popt, pcov = optim.curve_fit(f = exp_func, 
            xdata =np.array(range(len(cases))), ydata = np.array(cases) )
        final['exp_ratio'] = popt[1]
        lin_rate, p_value, y_hat_lin = get_linear(range(len(cases)), cases)
        final['lin_slope'] = lin_rate
        final['lin_p_value'] = p_value
        y_hat_exp = [exp_func(initial = popt[0], ratio = popt[1], x = x) 
            for x in range(len(cases))]
        final['exp_better_fit'] = ssr_is_sig(y = cases, y_hat_lin = y_hat_lin, y_hat_exp= y_hat_exp)
        sig_exp = exp_is_sig(x = range(len(cases)), y=cases)
        final['exp_p_value'] = exp_p_value(x = range(len(cases)), y=cases)
        return final

In [24]:
def get_confidence_interval(the_list):
    resamps = repeat_resample(sample = the_list)
    l =round( np.percentile(resamps, .05/2),2)
    h = round(np.percentile(resamps, 97.2),2)
    return l, h, not l <= 0 <= h

In [25]:
def get_diffs(the_dict):
    final = []
    for key in the_dict.keys():
        final.append((key, the_dict[key]['post']['exp_ratio'] - the_dict[key]['pre']['exp_ratio']))
    return final

In [26]:
def simple_bar(states, changes, plot_height = 450, plot_width = 450, title = 'RT change'):
    p = figure(x_range=states, plot_height = plot_height, plot_width = plot_width,
               title=title)
    p.vbar(x=states, top=changes, width=0.9)
    p.xgrid.grid_line_color = None
    p.xaxis.major_label_orientation = "vertical"
    return p

In [27]:
def binom_is_sig(l):
    lower_binomial, upper_binomial = binom.interval(.95, len(l), .5)
    num_decrease = len(list(filter(lambda x: x < 0, l)))
    return lower_binomial, upper_binomial, num_decrease, not lower_binomial <= num_decrease <= upper_binomial


In [28]:
def all_states_rates():
    rates_dict = {}
    df = pd.read_csv('data/states.csv')
    df['date'] =pd.to_datetime(df['date'])
    grid = []
    for i in get_state_mask_start():
        df_target = df[df['state'] == i[0]]
        pre_date = i[1] - datetime.timedelta(days = 15)
        post_date = i[1] + datetime.timedelta(days = 19)

        pre_cases = df_target[(df_target['date'] < i[1]) &
                    (df_target['date']> pre_date)
                             ].cases.tolist()
        post_cases = df_target[(df_target['date'] >= i[1] + datetime.timedelta(days = 5)) &
                    (df_target['date']< post_date)
                             ].cases.tolist()
        assert len(post_cases) == 14
        assert len(pre_cases) == 14
        rates_dict[i[0]] = {}
        rates_dict[i[0]]['pre'] = get_fit_info(pre_cases)
        rates_dict[i[0]]['post'] = get_fit_info(post_cases)
    return rates_dict

def do_states_rates(info):
    output_notebook()
    print_results(info)
    differences = sorted(get_diffs(info), key = lambda x: x[1])
    low, high, is_sig = get_confidence_interval([x[1] for x in differences])
    if is_sig:
        print('The confidence interval is between {l} and {h} and is significant'.format(
        l = low, h = high))
    else:
        print('The confidence interval is between {l} and {h} and is not significant'.format(
            l  = low, h = high))
    p_change = simple_bar(states = [x[0] for x in differences], 
               changes = [x[1] for x in differences], plot_height = 450, 
                          plot_width = 450)
    l_b, u_b, num_dec, b_sig = binom_is_sig([x[1] for x in differences])
    if b_sig:
        print('{n} lies outside {l} and {u} and is significant'.format(
            n = num_dec, l =l_b, u = u_b))
    else:
         print('{n} lies within {l} and {u} and is not significant'.format(
            n = num_dec, l =l_b, u = u_b))
    show(p_change)

info = all_states_rates()
do_states_rates(info)


Loading BokehJS ...

state		Before	After	Diff
=====		======	=====	====
Alabama		 1.04	 1.00	-0.04
Arizona		 1.05	 1.03	-0.02
California	 1.02	 1.03	 0.01
Connecticut	 0.97	 0.99	 0.02
Delaware	 1.05	 1.01	-0.04
Hawaii		 0.95	 0.96	 0.02
Illinois	 1.04	 0.98	-0.06
Kansas		 1.04	 0.99	-0.05
Kentucky	 1.07	 1.01	-0.06
Maine		 1.01	 0.98	-0.04
Maryland	 1.03	 1.01	-0.02
Massachusetts	 0.94	 0.99	 0.05
Michigan	 0.78	 1.02	 0.25
Nevada		 1.06	 1.01	-0.05
New Jersey	 1.06	 1.00	-0.06
New Mexico	 0.99	 1.00	 0.01
New York	 0.99	 0.93	-0.06
North Carolina	 0.99	 1.02	 0.03
Oregon		 1.02	 1.03	 0.01
Pennsylvania	 0.99	 0.97	-0.02
Rhode Island	 0.98	 0.98	-0.00
Texas		 1.05	 1.00	-0.05
Virginia	 1.03	 0.95	-0.08
Washington	 1.05	 1.03	-0.02
West Virginia	 1.12	 1.02	-0.10
The confidence interval is between -0.04 and 0.01 and is not significant
17 lies within 8.0 and 17.0 and is not significant


In [29]:
def all_states_poisson_rates():
    rates_dict = {}
    df = pd.read_csv('data/state_poisson.csv')
    df['date'] =pd.to_datetime(df['date'])
    grid = []
    for i in get_state_mask_start():
        df_target = df[df['state'] == i[0]]
        pre_date = i[1] - datetime.timedelta(days = 15)
        post_date = i[1] + datetime.timedelta(days = 14)

        pre_cases = df_target[(df_target['date'] < i[1]) &
                    (df_target['date']> pre_date)
                             ].cases.tolist()
        post_cases = df_target[(df_target['date'] >= i[1]) &
                    (df_target['date']< post_date)
                             ].cases.tolist()
        assert len(post_cases) == 14
        rates_dict[i[0]] = {}
        rates_dict[i[0]]['pre'] = get_fit_info(pre_cases)
        rates_dict[i[0]]['post'] = get_fit_info(post_cases)
    return rates_dict


In [30]:
info = all_states_poisson_rates()
do_states_rates(info)


Loading BokehJS ...

state		Before	After	Diff
=====		======	=====	====
Alabama		 1.03	 0.99	-0.04
Arizona		 1.08	 1.02	-0.05
California	 1.04	 1.04	-0.01
Connecticut	 1.01	 0.98	-0.04
Delaware	 1.00	 1.01	 0.01
Hawaii		 0.91	 0.93	 0.02
Illinois	 1.02	 0.99	-0.03
Kansas		 1.04	 1.00	-0.03
Kentucky	 1.06	 1.01	-0.04
Maine		 1.04	 1.02	-0.02
Maryland	 1.00	 1.02	 0.01
Massachusetts	 0.96	 0.97	 0.01
Michigan	 0.99	 1.04	 0.04
Nevada		 1.09	 1.01	-0.08
New Jersey	 1.01	 0.99	-0.02
New Mexico	 0.99	 1.01	 0.02
New York	 0.97	 0.94	-0.02
North Carolina	 1.02	 1.02	 0.00
Oregon		 1.04	 1.02	-0.01
Pennsylvania	 0.99	 0.98	-0.01
Rhode Island	 0.99	 0.96	-0.03
Texas		 1.03	 1.00	-0.03
Virginia	 1.00	 0.95	-0.05
Washington	 1.03	 1.03	-0.00
West Virginia	 1.09	 1.01	-0.08
The confidence interval is between -0.04 and -0.01 and is significant
18 lies outside 8.0 and 17.0 and is significant


In [31]:
def make_bar_over_time(labels, y, plot_height = 450, 
             plot_width = 450, title = None):
    x_range = [x[0] for x in labels]
    p = figure(x_range = x_range , y_range = (-3000, 3000), plot_height = plot_height, 
               plot_width = plot_width, title = title)
    for counter, i in enumerate(labels):
        if i[1] < .05:
            color = 'blue'
            legend_label = 'Significant'
        else:
            color = 'red'
            legend_label = 'Not Significant'
        p.vbar(x=[i[0]], top=y[counter] , line_width = 5, width = .5, 
               color = color, legend_label = legend_label)
    p.xaxis.major_label_orientation = "vertical"
    p.xgrid.grid_line_color = None
    p.legend.location = "bottom_right"

    return p
    #p.y_range 

In [32]:
def eval_rates_over_time():
    output_notebook()
    df_all = pd.read_csv('data/states.csv')
    df_all['date'] = pd.to_datetime(df_all['date'])
    start_date = datetime.datetime(2020,3, 1)
    data = []
    for i in get_state_mask_start_with_after():
        df = df_all[(df_all['state'] == i[0]) & (df_all['date'] >= start_date)]
        period_before = df[(df['date'] > i[1] - datetime.timedelta(days=15))\
                           & (df['date'] < i[1])
                          ]
        cases_before = period_before['cases'].tolist()
        period_after = df[(df['date'] > i[2])\
                           & (df['date'] < i[3])
                          ]
        cases_after = period_after['cases'].tolist()
        assert len(cases_before) == len(cases_after)
        stat_diff = ttest_ind(cases_before, cases_after)
        p_value = stat_diff.pvalue
        data.append((i[0], np.mean(cases_after - np.mean(cases_before)), p_value))
    data = sorted(data, key = lambda x: x[1])
    states = [x[0] for x in data]
    changes = [x[1] for x in data]
    labels = [(x[0], x[2]) for x in data]
    y = [x[1] for x in data]
    #p = simple_bar(states, changes, plot_height = 450, plot_width = 750, title = 'Change in Cases')
    #show(p)
    p = make_bar_over_time(labels, y, plot_height = 450, 
             plot_width = 750, title = 'cases over time')
    show(p)

        
eval_rates_over_time()


Loading BokehJS ...

In [33]:
def kansas():
    
    pass
kansas()